In [2]:
import findspark

In [3]:
findspark.init()

In [4]:
import pyspark

In [6]:
import pyspark.sql

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [11]:
# Read The Dataset
training = spark.read.csv('C://test1.csv',header=True,inferSchema=True)

In [12]:
training.show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 30000|
|Sudhansh| 30|         8| 25000|
|   Sunny| 29|         4| 20000|
|    Paul| 24|         3| 20000|
|  Harsha| 21|         1| 15000|
| Shubham| 23|         2| 18000|
+--------+---+----------+------+



In [13]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [14]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [17]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")

In [18]:
output=featureassembler.transform(training)

In [19]:
output.show()

+--------+---+----------+------+--------------------+
|    Name|Age|Experience|Salary|Independent Features|
+--------+---+----------+------+--------------------+
|   Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhansh| 30|         8| 25000|          [30.0,8.0]|
|   Sunny| 29|         4| 20000|          [29.0,4.0]|
|    Paul| 24|         3| 20000|          [24.0,3.0]|
|  Harsha| 21|         1| 15000|          [21.0,1.0]|
| Shubham| 23|         2| 18000|          [23.0,2.0]|
+--------+---+----------+------+--------------------+



In [20]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [21]:
finalized_data=output.select("Independent Features","Salary")

In [22]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [29]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="Independent Features", labelCol="Salary")
regressor = regressor.fit(train_data)

In [30]:
### Coefficients
regressor.coefficients

DenseVector([-0.0, 1666.6667])

In [31]:
### Intercept
regressor.intercept

13333.333333333807

In [32]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [33]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|16666.666666666693|
|          [24.0,3.0]| 20000| 18333.33333333336|
|          [30.0,8.0]| 25000|26666.666666666664|
+--------------------+------+------------------+



In [34]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1555.5555555555366, 2444444.4444443868)